In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [18]:
path = r"C:\\Users\\USER\\Desktop\\font_original" #원본 이미지 경로 
copy_path = r"C:\\Users\\USER\\Desktop\\font_classifier" #복사 및 분류된 이미지 경로
#font_classifier 안에 모든 글꼴 이미지들이 다 저장되어 있음 (따로 하위폴더 없음)
#노트북 경로

In [19]:
import splitfolders

splitfolders.ratio(path, output=copy_path, seed = 43, ratio = (.7,.3))

Copying files: 55860 files [03:07, 298.31 files/s]


In [28]:
#각 train, test, vali 폴더 안에서 글꼴별로 구별되어 있는 걸 풀어서 섞고, 글꼴별 폴더 삭제하기

import shutil
import os

font_list = ["01","02","03","04","05"]
folder_list = ["val","train"]

for folder in folder_list :


    for font in font_list :
        currentPath = copy_path + f'\\{folder}'+ f'\\{font}'

        trainFileList = os.listdir(currentPath)
        trainFileList = [x for x in trainFileList if 'png' in x]

        for image in trainFileList :
            try :
                shutil.move(currentPath + f'\\{image}', copy_path + f'\\{folder}' + f'\\{image}')
            except :
                pass
    
        os.rmdir(currentPath)


In [29]:
import shutil
import os

for folder in folder_list :
    currentPath = copy_path + f'\\{folder}'

    trainFileList = os.listdir(currentPath)
    trainFileList = [x for x in trainFileList if 'png' in x]

    label_list = list(set(map(lambda x : x[:-7], trainFileList))) 
    
    try:
        for key in label_list: # 글자마다 폴더가 생김
            os.makedirs(currentPath + f'\\{key}')
    except:
        pass

    # 여기까지는 정상적으로 처리 됨.

    # font_classifier 폴더에 있던 마구자비로 섞여있던 이미지 데이터들이 train 폴더 안에 있는 글자 폴더별로 옮겨짐
    for file in trainFileList:
        label = file.split('.')[0]
        targetPath = currentPath + f'\\{label}'
        try :
            shutil.move(currentPath+f'\\{file}', targetPath+f'\\{file}')
        except:
            pass


#여기까지 자료 준비 완료 (글자별로 분류된 상태)

In [20]:
print(len(os.listdir(copy_path + '\\val')))
print(len(os.listdir(copy_path + '\\train')))

5
5


### 데이터 준비 완료 -----------

In [21]:
trainDataGen = ImageDataGenerator(rescale=1./255)

In [22]:
currentPath = copy_path + '\\train'

In [23]:
trainGenSet = trainDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 39100 images belonging to 5 classes.


In [24]:
valDataGen = ImageDataGenerator(rescale=1./255)

In [25]:
currentPath = copy_path + '\\val'

In [26]:
valGenSet = valDataGen.flow_from_directory(
    currentPath,
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 16760 images belonging to 5 classes.


In [27]:
from keras.layers import *
from sklearn.metrics import accuracy_score

In [28]:
# 모델 구성
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
    
model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))  #18로 되어있었는데, 출력 라벨을 바꿔야 할 거 같은 구글링 결과가 나와서 바꿔봄...! => 성공!!! (validation, train의 라벨이 다 같아야 함!!)

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 7, 7, 128)        

In [30]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(
    trainGenSet,
    steps_per_epoch=40,
    epochs=400,
    validation_data=valGenSet,
    validation_steps=20
)

Epoch 1/400
40/40 [==============================] - 47s 1s/step - loss: 1.1162 - accuracy: 0.5129 - val_loss: 0.4780 - val_accuracy: 0.7125
Epoch 2/400
40/40 [==============================] - 45s 1s/step - loss: 0.4091 - accuracy: 0.7762 - val_loss: 0.3118 - val_accuracy: 0.8055
Epoch 3/400
40/40 [==============================] - 47s 1s/step - loss: 0.3897 - accuracy: 0.8023 - val_loss: 0.3136 - val_accuracy: 0.8211
Epoch 4/400
40/40 [==============================] - 34s 848ms/step - loss: 0.2863 - accuracy: 0.8453 - val_loss: 0.2128 - val_accuracy: 0.9062
Epoch 5/400
40/40 [==============================] - 25s 628ms/step - loss: 0.2254 - accuracy: 0.8973 - val_loss: 0.2822 - val_accuracy: 0.8555
Epoch 6/400
40/40 [==============================] - 26s 663ms/step - loss: 0.2053 - accuracy: 0.9137 - val_loss: 0.1560 - val_accuracy: 0.9344
Epoch 7/400
40/40 [==============================] - 24s 594ms/step - loss: 0.1413 - accuracy: 0.9379 - val_loss: 0.0805 - val_accuracy: 0.9688
E

In [1]:
model.summary()

NameError: name 'model' is not defined

### 모델 돌리기 성공